In [7]:
import pandas as pd
import numpy as np
import shin
from sklearn.metrics import *

In [8]:
def shin_proba(data_odd):
  data_odd_copy = data_odd.copy() 

  for i in range(data_odd.shape[0]):
    data_odd_copy.iloc[i] = np.array(shin.calculate_implied_probabilities(data_odd_copy.iloc[i]))

  return data_odd_copy

In [9]:
temp = pd.concat([pd.read_csv("~/new_work/football/data_scraper/base_data.csv")])
temp['Date'] = pd.to_datetime(temp['Kick Off Time'])
temp = temp.sort_values(by=['Date'], ascending=True)
temp

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jokerlaugh/new_work/football/data_scraper/base_data.csv'

In [70]:
temp = pd.concat([pd.read_csv("~/new_work/football/data_scraper/extend_data.csv")])
temp['Date'] = pd.to_datetime(temp['Kick Off Time'])
temp = temp.sort_values(by=['Date'], ascending=True)
temp

In [71]:
#start form 2024-10-24
data = pd.concat([pd.read_csv("~/new_work/football/data_scraper/base_data.csv"),
                  pd.read_csv("~/new_work/football/data_scraper/extend_data.csv")],
                 axis=0)
'''data = pd.concat([pd.read_csv("~/new_work/football/data_scraper/extend_data.csv")],
                 axis=0)'''
data = data[['Kick Off Time', 'Home Result', 'Away Result', 'Home Odds', 'Draw Odds', 'Away Odds']]
data['FTR'] = 0
data.loc[data['Home Result'] > data['Away Result'], 'FTR'] = 0
data.loc[data['Home Result'] == data['Away Result'], 'FTR'] = 1
data.loc[data['Home Result'] < data['Away Result'], 'FTR'] = 2
data['Date'] = pd.to_datetime(data['Kick Off Time'])
data = data.sort_values(by=['Date'], ascending=True)
data = data.reset_index(drop=True)
data = data.rename(columns={'Home Odds': 'B365H',
                                    'Draw Odds': 'B365D',
                                    'Away Odds': 'B365A',
                                    'FTR': 'FTR'})
#data = data.dropna()

#'PSH', 'PSD', 'PSA'
B365_odd = data[['B365H', 'B365D', 'B365A']]
data[['B365H', 'B365D', 'B365A']] = shin_proba(data[['B365H', 'B365D', 'B365A']])

#data = data.dropna()
number = 16384
#data = data.iloc[(-number):]
#B365_odd = B365_odd.iloc[(-number):]
number = round(data.shape[0]*0.2)
#number = 2048

In [72]:
data

In [73]:
import numpy as np

def project_to_simplex(y):
    """
    Projects a vector y onto the probability simplex.

    Parameters:
    y (np.ndarray): Input vector of shape (D,).

    Returns:
    np.ndarray: Projected vector on the probability simplex.
    """
    y = y.flatten()
    # Ensure y is a NumPy array of type float
    y = np.asarray(y, dtype=np.float64)
    
    # Sort y in descending order
    u = np.sort(y)[::-1]
    
    # Compute the cumulative sum of u
    cssv = np.cumsum(u)
    
    # Find the rho value using the condition u_j + (1 - cssv_j) / j > 0
    rho = np.nonzero(u + (1 - cssv) / np.arange(1, len(u)+1) > 0)[0]
    
    if len(rho) == 0:
        rho = 0
    else:
        rho = rho[-1]
    
    # Compute the Lagrange multiplier lambda
    lambda_val = (1 - cssv[rho]) / (rho + 1)
    
    # Compute the projected vector
    w = np.maximum(y + lambda_val, 0)
    
    return w

In [74]:
def kalman_opt(parameter):
    F = np.eye(6)
    '''Q = np.array([[parameter[0],0,0,0,0,0],
                  [0,parameter[1],0,0,0,0],
                  [0,0,parameter[2],0,0,0],
                  [0,0,0,parameter[3],0,0],
                  [0,0,0,0,parameter[4],0],
                  [0,0,0,0,0,parameter[5]]])'''
    '''Q = np.array([[0,0,0,0,0,0],
                  [0,0,0,0,0,0],
                  [0,0,0,0,0,0],
                  [0,0,0,parameter[0],0,0],
                  [0,0,0,0,parameter[1],0],
                  [0,0,0,0,0,parameter[2]]])'''
    gamma = parameter[0]
    #q_values = parameter[1:]
    Q = np.eye(6)
    #Q = np.diag(q_values)
    #Q = Q*parameter[0]
    x = np.array([[1],
                  [1],
                  [1],
                  [0],
                  [0],
                  [0]])
    P = np.eye(6)
    '''P = np.array([[0,0,0,0,0,0],
                  [0,0,0,0,0,0],
                  [0,0,0,0,0,0],
                  [0,0,0,1,0,0],
                  [0,0,0,0,1,0],
                  [0,0,0,0,0,1]])'''
    
    FTR_dummies = pd.get_dummies(data['FTR'].iloc[:number])
    test_record = np.empty((0,3))
    for i in range(number):
        p1 = data['B365H'].iloc[i]
        p2 = data['B365D'].iloc[i]
        p3 = data['B365A'].iloc[i]
        H = np.array([[p1,0,0,1,0,0],
                      [0,p2,0,0,1,0],
                      [0,0,p3,0,0,1]])
        
        r1 = data['B365H'].iloc[i]*(1-data['B365H'].iloc[i])
        r2 = data['B365D'].iloc[i]*(1-data['B365D'].iloc[i])
        r3 = data['B365A'].iloc[i]*(1-data['B365A'].iloc[i])
        r12 = -data['B365H'].iloc[i]*data['B365D'].iloc[i]*0
        r13 = -data['B365H'].iloc[i]*data['B365A'].iloc[i]*0
        r23 = -data['B365D'].iloc[i]*data['B365A'].iloc[i]*0
        R = np.array([[r1,r12,r13],
                      [r12,r2,r23],
                      [r13,r23,r3]])
    
        x_pred = F @ x
        P_pred = F @ P @ F.T + Q
    
        prediction = (H @ x_pred).T
        '''if(min(prediction[0]) < 0):
            prediction = prediction - min(prediction[0])
        test_record = np.vstack([test_record, prediction/prediction.sum()])'''
        prediction = project_to_simplex(prediction).reshape(-1, 3)
        test_record = np.vstack([test_record, prediction])
    
        z = np.array([[FTR_dummies[0].iloc[i]],
                      [FTR_dummies[1].iloc[i]],
                      [FTR_dummies[2].iloc[i]]])
        K = P_pred @ H.T @ np.linalg.inv(H @ P_pred @ H.T + R)
        x = x_pred + K @ (z - H @ x_pred)
        P = (np.eye(6) - K @ H) @ P_pred @ (np.eye(6) - K @ H).T + K @ R @ K.T
    
        '''x = np.array([[1],
                      [1],
                      [1],
                      [x[3][0]],
                      [x[4][0]],
                      [x[5][0]]])'''
        e_k = x-x_pred
        Q = gamma*Q + (1-gamma)* (e_k @ e_k.T)
    
    return log_loss(FTR_dummies, test_record)
    #return multiclass_focal_log_loss(FTR_dummies, test_record)

from scipy.optimize import *
opt_result = minimize(kalman_opt,
                      x0=[0.5],
                      method="Nelder-Mead",
                      options={'adaptive':True, 'disp':True},
                  bounds=[(0, 1)])
'''opt_result = differential_evolution(kalman_opt,
                  bounds=[(0, 1),(0, 1),(0, 1),(0, 1),(0, 1),(0, 1)],
                                   workers=-1)'''
'''opt_result = shgo(kalman_opt,
                  bounds=[(0, 1),(0, 1),(0, 1),(0, 1),(0, 1),(0, 1)],
                                   workers=-1)'''
'''opt_result = differential_evolution(kalman_opt,
                  bounds=[(0, 1)],
                                    init='sobol',
                                   workers=-1)'''
print(opt_result)

In [123]:
F = np.eye(6)
Q = np.eye(6)
#Q = Q*8.594e-05
gamma = opt_result['x'][0]
#gamma = 0.5
x = np.array([[1],
              [1],
              [1],
              [0],
              [0],
              [0]])
P = np.eye(6)

FTR_dummies = pd.get_dummies(data['FTR'])
test_record = np.empty((0,3))
for i in range(data.shape[0]):
    '''if(i+128 > data.shape[0]-1):
        break'''
    
    p1 = data['B365H'].iloc[i]
    p2 = data['B365D'].iloc[i]
    p3 = data['B365A'].iloc[i]
    H = np.array([[p1,0,0,1,0,0],
                  [0,p2,0,0,1,0],
                  [0,0,p3,0,0,1]])
    
    r1 = data['B365H'].iloc[i]*(1-data['B365H'].iloc[i])
    r2 = data['B365D'].iloc[i]*(1-data['B365D'].iloc[i])
    r3 = data['B365A'].iloc[i]*(1-data['B365A'].iloc[i])
    r12 = -data['B365H'].iloc[i]*data['B365D'].iloc[i]*0
    r13 = -data['B365H'].iloc[i]*data['B365A'].iloc[i]*0
    r23 = -data['B365D'].iloc[i]*data['B365A'].iloc[i]*0
    R = np.array([[r1,r12,r13],
                  [r12,r2,r23],
                  [r13,r23,r3]])

    x_pred = F @ x
    P_pred = F @ P @ F.T + Q

    if(i+128 <= data.shape[0]-1):
        p1_delay = data['B365H'].iloc[i+128]
        p2_delay = data['B365D'].iloc[i+128]
        p3_delay = data['B365A'].iloc[i+128]
        H_delay = np.array([[p1_delay,0,0,1,0,0],
                      [0,p2_delay,0,0,1,0],
                      [0,0,p3_delay,0,0,1]])
        prediction = (H_delay @ x_pred).T
        prediction = project_to_simplex(prediction).reshape(-1, 3)
        test_record = np.vstack([test_record, prediction.copy()])
    

    z = np.array([[FTR_dummies[0].iloc[i]],
                  [FTR_dummies[1].iloc[i]],
                  [FTR_dummies[2].iloc[i]]])
    K = P_pred @ H.T @ np.linalg.inv(H @ P_pred @ H.T + R)
    x = x_pred + K @ (z - H @ x_pred)
    P = (np.eye(6) - K @ H) @ P_pred @ (np.eye(6) - K @ H).T + K @ R @ K.T
    
    e_k = x-x_pred
    Q = gamma*Q + (1-gamma)* (e_k @ e_k.T)

    '''x = np.array([[1],
              [1],
              [1],
              [0],
              [0],
              [0]])'''

In [124]:
prediction.copy()

In [125]:
x

In [126]:
x_pred

In [127]:
def nearest_positive_definite(A, epsilon=1e-32, max_iterations=1024):
    B = (A + A.T) / 2
    _, s, V = np.linalg.svd(B)
    H = V.T @ np.diag(s) @ V
    A_plus = (B + H) / 2

    for _ in range(max_iterations):
        A_prev = A_plus
        A_plus = (A_prev + A_prev.T) / 2
        eigenvalues, eigenvectors = np.linalg.eigh(A_plus)
        eigenvalues_positive = np.maximum(eigenvalues, epsilon)
        A_plus = eigenvectors @ np.diag(eigenvalues_positive) @ eigenvectors.T

        if np.allclose(A_plus, A_prev, atol=1e-32):
            break

    return A_plus

In [128]:
def downside_risk(returns, risk_free=0):
    adj_returns = returns - risk_free
    sqr_downside = np.square(np.clip(adj_returns, np.NINF, 0))
    return np.sqrt(np.nanmean(sqr_downside) * 252)


def sortino(returns, risk_free=0):
    adj_returns = returns - risk_free
    drisk = downside_risk(adj_returns)

    if drisk == 0:
        return np.nan

    return (np.nanmean(adj_returns) * np.sqrt(252)) / drisk

In [129]:
import numpy as np
from scipy.optimize import *

def kelly_criterion_three_outcomes(p, b):
    """
    Calculate the Kelly optimal fractions for three mutually exclusive outcomes.

    Parameters:
    - p: List or array of probabilities [p_A, p_B, p_C]. Should sum to 1.
    - b: List or array of net odds [b_A, b_B, b_C].

    Returns:
    - Optimal fractions [f_A, f_B, f_C]
    - Total bet fraction
    """

    # Convert to numpy arrays
    p = np.array(p, dtype=np.float64)
    b = np.array(b, dtype=np.float64)
    b = b-1

    # Normalize probabilities to ensure they sum to 1
    if not np.isclose(p.sum(), 1):
        p = p / p.sum()
        #print("Probabilities normalized to sum to 1.")

    # Validate inputs
    if any(p <= 0):
        raise ValueError("All probabilities must be positive.")
    if any(b <= 0):
        raise ValueError("All odds must be positive.")

    # Initial guess: Proportion based on probabilities
    initial_guess = p.copy()*0

    # Define the objective function: Negative expected log wealth
    def objective(f):
        f_A, f_B, f_C = f
        wealth_A = 1 + b[0]*f_A - f_B - f_C
        wealth_B = 1 + b[1]*f_B - f_A - f_C
        wealth_C = 1 + b[2]*f_C - f_A - f_B

        # To handle log(0) or negative, return a large penalty
        if wealth_A <= 0 or wealth_B <= 0 or wealth_C <= 0:
            return 1e10

        expected_log_wealth = p[0]*np.log(wealth_A) + p[1]*np.log(wealth_B) + p[2]*np.log(wealth_C)
        return -expected_log_wealth  # Negative for minimization

    # Constraints:
    constraints = [
        {
            'type': 'ineq',
            'fun': lambda f: 0.9999 - np.sum(f)  # 1 - (f_A + f_B + f_C) >= 0
        },
        {
            'type': 'ineq',
            'fun': lambda f: 1 + b[0]*f[0] - f[1] - f[2]  # wealth_A > 0
        },
        {
            'type': 'ineq',
            'fun': lambda f: 1 + b[1]*f[1] - f[0] - f[2]  # wealth_B > 0
        },
        {
            'type': 'ineq',
            'fun': lambda f: 1 + b[2]*f[2] - f[0] - f[1]  # wealth_C > 0
        }
    ]

    # Bounds: 0 <= f_A, f_B, f_C <=1
    bounds = [(0, 1) for _ in range(3)]

    # Perform the optimization
    result = minimize(
        objective,
        initial_guess,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints,
        options={'disp': False, 'ftol':1e-9, 'maxiter':1000}
    )
    '''result = shgo(
        objective,
        bounds=bounds,
        constraints=constraints,
        workers=1
    )'''

    if not result.success:
        # Provide detailed information about the failure
        print("Optimization Failed!")
        print("Message:", result.message)
        # To provide meaningful constraint values, attempt to evaluate constraints at the initial guess
        initial_constraints = [constraint['fun'](initial_guess) for constraint in constraints]
        print("Initial Constraint Values:")
        for i, val in enumerate(initial_constraints):
            print(f"  Constraint {i+1}: {val:.6f} (should be >= 0)")
        # Suggest using fractional Kelly if all else fails
        print("\nPossible Reasons:")
        print("- The odds are unfavorable given the probabilities.")
        print("- The optimizer cannot find feasible bet fractions with the given constraints.")
        print("\nSuggestions:")
        print("- Review and adjust the probabilities and odds to ensure feasibility.")
        print("- Consider using a fractional Kelly strategy to reduce bet sizes.")
        print("- Ensure that the sum of probabilities is 1 and all odds are positive.")
        raise ValueError("Optimization failed: " + result.message)

    optimal_fractions = result.x

    # Ensure fractions are non-negative and sum does not exceed 1
    optimal_fractions = np.clip(optimal_fractions, 0, 1)
    total_bet = optimal_fractions.sum()

    return optimal_fractions

In [130]:
'''{
            'type': 'ineq',
            'fun': lambda f: 1-( p[0]*(b[0]*f[0] - f[1] - f[2] + f[3])**(-2)+\
            p[1]*(b[1]*f[1] - f[0] - f[2] + f[3])**(-2)+\
            p[2]*(b[2]*f[2] - f[0] - f[1] + f[3])**(-2) )
        }'''

In [131]:
test_record[number-128:].shape

In [132]:
B365_odd[number:].shape

In [133]:
from tqdm import *
from joblib import Parallel, delayed

kelly = test_record[number-128:] - (1 - test_record[number-128:]) / (B365_odd[number:].to_numpy() - 1)
def temp_f(i):
    return kelly_criterion_three_outcomes(test_record[number-128:][i]+1e-5, B365_odd[number:].to_numpy()[i])
'''for i in tqdm(range(kelly.shape[0])):
    kelly[i] = kelly_criterion_three_outcomes(test_record[number-128:][i]+1e-5, B365_odd[number:].to_numpy()[i])
'''
kelly = Parallel(n_jobs=-1)(delayed(temp_f)(i) for i in range(test_record[number-128:].shape[0]))
kelly = np.vstack(kelly)

In [ ]:
# global_kelly_copy = kelly.copy()
def fraction_opt(f):
    #global kelly
    kelly = global_kelly_copy * f[0]  # or kelly * 0.3 if preferred
    kelly = np.clip(kelly, 0, 1)
    
    # Select the choice with the maximum Kelly fraction
    #max_kelly = np.max(kelly, axis=1, keepdims=True)
    #choice = (kelly == max_kelly) & (kelly > 0)
    choice = (kelly > 0)
    
    # Convert 'FTR' to one-hot encoding
    actual_results = pd.get_dummies(data[['FTR']].iloc[number:], columns=['FTR']).to_numpy()
    
    # Check if the chosen bet matches the actual result
    # Revised to ensure any match is considered correct
    is_correct = (choice & actual_results)
    
    # Extract the odds for the chosen outcomes
    #chosen_indices = kelly.argmax(axis=1)
    #chosen_odds = B365_odd.to_numpy()[np.arange(len(kelly)), chosen_indices]
    chosen_odds = B365_odd.to_numpy()[np.arange(number-128, number-128+len(kelly)), :]
    
    # Calculate returns
    #returns = chosen_odds * kelly.max(axis=1) * is_correct - kelly.max(axis=1)
    returns = (chosen_odds * kelly * is_correct).sum(axis=1) - kelly.sum(axis=1)

    return -np.log10((1+returns)).sum()

opt_result_fraction = differential_evolution(fraction_opt,
                                             bounds=[(0, 0.9)],
                                             init='sobol',
                                             maxiter=10000,
                                             workers=-1)
print(opt_result_fraction)

global_kelly_copy = global_kelly_copy * opt_result_fraction['x'][0]  # or kelly * 0.3 if preferred
global_kelly_copy = np.clip(global_kelly_copy, 0, 1)

# Select the choice with the maximum Kelly fraction
#max_kelly = np.max(kelly, axis=1, keepdims=True)
#choice = (kelly == max_kelly) & (kelly > 0)
choice = (global_kelly_copy > 0)

# Convert 'FTR' to one-hot encoding
actual_results = pd.get_dummies(data[['FTR']].iloc[number:], columns=['FTR']).to_numpy()

# Check if the chosen bet matches the actual result
# Revised to ensure any match is considered correct
is_correct = (choice & actual_results)

# Extract the odds for the chosen outcomes
#chosen_indices = kelly.argmax(axis=1)
#chosen_odds = B365_odd.to_numpy()[np.arange(len(kelly)), chosen_indices]
chosen_odds = B365_odd.to_numpy()[np.arange(number-128, number-128+len(global_kelly_copy)), :]

# Calculate returns
#returns = chosen_odds * kelly.max(axis=1) * is_correct - kelly.max(axis=1)
returns = (chosen_odds * global_kelly_copy * is_correct).sum(axis=1) - global_kelly_copy.sum(axis=1)

In [137]:
pd.get_dummies(data[['FTR']].iloc[number:], columns=['FTR'])

In [135]:
np.log10(1+returns).sum()

In [136]:
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

plt.plot(returns)

In [88]:
kelly.sum(axis=1)

In [89]:
returns

In [90]:
kelly.sum(axis=1)

In [91]:
kelly

In [92]:
B365_odd

In [93]:
kelly.sum(axis=1)

In [94]:
B365_odd.to_numpy()[np.arange(len(kelly)), :]

In [95]:
is_correct

In [96]:
temp = np.log((1+returns))
mu = temp.mean()*1
var = temp.var()*1

m = np.exp(mu+var/2)
print(m-1)
print((np.exp(var)-1)*np.exp(2*m+var))

#print(mu*64)
#print(temp.std()*np.sqrt(64))

In [118]:
(1.05**2)*(30)

In [97]:
returns.shape

In [98]:
np.log10(1+returns).sum()

In [99]:
np.log10(1+returns).cumsum()

In [100]:
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

#temp = pd.DataFrame(np.log10((1+returns).cumprod()))
temp = pd.DataFrame(np.log10(1+returns).cumsum() )
temp.index = pd.to_datetime(data.iloc[number:]['Date'].values)
#temp = (temp.iloc[0:128*4])
#temp
temp.plot(figsize=(16,10))

#plt.savefig('test.png', dpi=300)

In [101]:
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

temp = pd.DataFrame((1+returns))
#temp.index = pd.to_datetime(data.iloc[number:]['Date'].values)
#temp = (temp.iloc[0:128*4])
#temp.plot(figsize=(16,10))

# If you have cumulative returns
cumulative_returns = np.log10(temp[0]).cumsum()
cumulative_returns = 10**cumulative_returns

# Calculate the running maximum
running_max = cumulative_returns.cummax()

# Calculate drawdown
drawdown = (cumulative_returns - running_max) / running_max

# Add drawdown to the DataFrame
temp['Drawdown'] = drawdown

temp.index = pd.to_datetime(data.iloc[number:]['Date'].values)
temp['Drawdown'].plot(figsize=(16*1,10*1))

plt.savefig('test.png', dpi=300)

In [102]:
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

temp = pd.DataFrame((1+returns))
#temp.index = pd.to_datetime(data.iloc[number:]['Date'].values)
#temp = (temp.iloc[0:128*4])
#temp.plot(figsize=(16,10))

# If you have cumulative returns
cumulative_returns = np.log10(temp[0]).cumsum()
#cumulative_returns = 10**cumulative_returns

# Calculate the running maximum
running_max = cumulative_returns.cummax()

# Calculate drawdown
drawdown = (cumulative_returns) - running_max

# Add drawdown to the DataFrame
temp['Drawdown'] = drawdown

temp.index = pd.to_datetime(data.iloc[number:]['Date'].values)
temp['Drawdown'].plot(figsize=(16*1,10*1))

plt.savefig('test.png', dpi=300)

In [103]:
cumulative_returns

In [104]:
temp['Drawdown'].min()

In [105]:
10**(-0.2)

In [106]:
data

In [107]:
odd = np.array([
2.21,

3.35,

2.62])
shin_p = shin.calculate_implied_probabilities(odd)
p1 = shin_p[0]
p2 = shin_p[1]
p3 = shin_p[2]
H = np.array([[p1,0,0,1,0,0],
              [0,p2,0,0,1,0],
              [0,0,p3,0,0,1]])

x_pred = F @ x
prediction = (H @ x_pred).T
prediction = project_to_simplex(prediction).reshape(-1, 3)
#prediction
#kelly = prediction - (1-prediction)/(odd-1)
#kelly = kelly*opt_result_fraction['x'][0]
temp_kelly = kelly_criterion_three_outcomes(prediction.flatten()+1e-8, odd)
temp_kelly = temp_kelly*opt_result_fraction['x'][0]


np.set_printoptions(suppress=True)
#principle = 971
principle = 371.30
print(np.clip(temp_kelly, 0, 1)*principle)

In [ ]:
92.37270152*(5.00-1) - 271.73048245 - 74.15233552

In [ ]:
prediction

In [ ]:
x_pred

In [ ]:
(np.clip(temp_kelly, 0, 1)).sum()

In [ ]:
opt_result_fraction['x'][0]